In [5]:
from collections import defaultdict
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk.tokenize import regexp_tokenize, word_tokenize, RegexpTokenizer
from nltk import pos_tag
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
import matplotlib.pyplot as plt
import string
import re
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, ConfusionMatrixDisplay

# Notice that these vectorizers are from `sklearn` and not `nltk`!
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer,\
HashingVectorizer

import nltk
nltk.download('wordnet')
nltk.download('tagsets')
nltk.download('averaged_perceptron_tagger')


[nltk_data] Downloading package wordnet to /Users/jayiraj/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package tagsets to /Users/jayiraj/nltk_data...
[nltk_data]   Package tagsets is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/jayiraj/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [6]:
df= pd.read_csv('../data/judge-1377884607_tweet_product_company.csv', encoding='latin1')



In [8]:
df.columns

Index(['tweet_text', 'emotion_in_tweet_is_directed_at',
       'is_there_an_emotion_directed_at_a_brand_or_product'],
      dtype='object')

In [11]:
df['tweet_text']

0       .@wesley83 I have a 3G iPhone. After 3 hrs twe...
1       @jessedee Know about @fludapp ? Awesome iPad/i...
2       @swonderlin Can not wait for #iPad 2 also. The...
3       @sxsw I hope this year's festival isn't as cra...
4       @sxtxstate great stuff on Fri #SXSW: Marissa M...
                              ...                        
9088                        Ipad everywhere. #SXSW {link}
9089    Wave, buzz... RT @mention We interrupt your re...
9090    Google's Zeiger, a physician never reported po...
9091    Some Verizon iPhone customers complained their...
9092    Ï¡Ïàü_ÊÎÒ£Áââ_£â_ÛâRT @...
Name: tweet_text, Length: 9093, dtype: object

In [ ]:
df['is_there_an_emotion_directed_at_a_brand_or_product'].value_counts()

In [ ]:
df['emotion_in_tweet_is_directed_at'].value_counts()

In [ ]:
# Rename the columns
df = df.rename(columns={
    'emotion_in_tweet_is_directed_at': 'products',
    'is_there_an_emotion_directed_at_a_brand_or_product': 'emotions'
})

# Check the updated DataFrame
print(df.head())


In [ ]:
df['emotions'].value_counts()

In [ ]:
# Replace the value in the 'emotions' column
df['emotions'] = df['emotions'].replace('No emotion toward brand or product', 'No emotion')

# Check the updated distribution
print(df['emotions'].value_counts())

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Set the style for seaborn
sns.set(style="whitegrid")

# Plot the updated distribution of emotions
plt.figure(figsize=(8, 6))
sns.countplot(x='emotions', data=df)
plt.title('Updated Distribution of Emotions Directed at a Brand or Product')
plt.xlabel('Emotion Category')
plt.ylabel('Count')

plt.xticks(rotation=45, ha='right')
plt.show()

In [ ]:
# Plot the updated distribution of emotions
plt.figure(figsize=(8, 6))
#sns.countplot(x='products', data=df)
sns.countplot(df['products'])
plt.title('Updated Distribution of Emotions Directed at a Brand or Product')
plt.xlabel('Emotion Category')
plt.ylabel('Count')

plt.xticks(rotation=45, ha='right')
plt.show()

In [ ]:
df.info()

In [ ]:
df['tweet_text'][:50]

In [ ]:
df['products'].value_counts()

In [ ]:
df['products'].isna().sum()

In [ ]:
df[df['products'].isna()].head(50)

In [ ]:
apple_products = [
    'iPhone',
    'iPad',
    'MacBook',
    'iMac',
    'Mac Mini',
    'Apple Watch',
    'AirPods',
    'HomePod',
]

android_products = [
    'Samsung Galaxy',
    'Google Pixel',
    'OnePlus',
    'LG G series',
    'Huawei P series',
]

# List of Google products
google_products = [
    'Google Pixel',
    'Google Home',
    'Google Nest',
    'Chromecast',
    'Pixelbook',
]

In [ ]:
# Function to update products based on the tweet text
def update_products(tweet_text):
    if pd.notna(tweet_text):
        if any(product.lower() in tweet_text.lower() for product in apple_products):
            return 'Apple'
        elif any(product.lower() in tweet_text.lower() for product in android_products):
            return 'Android'
        elif any(product.lower() in tweet_text.lower() for product in google_products):
            return 'Google'
    return pd.NA  # or any value you want for other cases


In [ ]:
df[df['products'].isna()].head(50) 

In [ ]:
import re

#filter to dectect non ascci caracters and return boolean value in ascii
df['ascii'] = df['tweet_text'].apply(lambda x: bool(re.search(r'[:;]-?[\)DPO\(]', str(x))))

# Display the result
print(df[['tweet_text', 'ascii']])

In [ ]:
df['ascii'].value_counts()

In [ ]:
df[df['ascii']].head(50)

In [ ]:
dfNaN = df[(df['products'].isna()) & (df['emotions'] == 'No emotion')].copy()

# Display the new DataFrame
dfNaN

In [ ]:
# Define the positive emoticons
positive_emoticons = [":')", "XP", "xD", "8)", ":-P", "=]", ":p", ";P", ":P", "XD", "(:", "=)", ":]", ";)", "=D", ";-)", ";D", ":-D", ":-)", ":)", ":D"]

# Define the negative emoticons
negative_emoticons = [":(", ":'(", ":-(", ";(", ">:(", ":/", ":-/", "D:", ":\\", "DX"]

#create a new column new_e to test the column update
dfNaN['new_e'] = None

# Update 'new_e' based on positive and negative emoticons
positive_pattern = '|'.join(map(re.escape, positive_emoticons))
negative_pattern = '|'.join(map(re.escape, negative_emoticons))

# Use fillna to replace NaN values in 'tweet_text' with an empty string
dfNaN['tweet_text'] = dfNaN['tweet_text'].fillna('')

dfNaN.loc[dfNaN['tweet_text'].str.contains(positive_pattern), 'new_e'] = 'Positive emotion'
dfNaN.loc[dfNaN['tweet_text'].str.contains(negative_pattern), 'new_e'] = 'Negative emotion'

# Display the updated DataFrame
dfNaN

In [ ]:
import emoji
import regex as re

# Function to check if a text contains miscellaneous characters
def contains_misc_characters(text):
    if isinstance(text, str):
        pattern = re.compile(r'[\x00-\x7F]+')  # Matches non-ASCII characters
        return not pattern.match(text)
    return False

# Apply the function to each line in the 'tweet_text' column and create a new 'tweet_misc' column
df['tweet_misc'] = df['tweet_text'].apply(contains_misc_characters)

# Display the result
print(df[['tweet_text', 'tweet_misc']])

In [ ]:
df['tweet_misc'].value_counts()

In [ ]:
df[df['tweet_misc']]

In [ ]:
df['products'].isna().sum()